In [1]:
import pandas as pd
import numpy as np

In [2]:
spon_df = pd.read_csv('jan_spon_info.csv').drop('Unnamed: 0', axis=1)


In [3]:
spon_df.head()

,trial_id,protocol_country,sponsor_name,sponsor_country,sponsor_status
0,2011-000593-54,IT,AZIENDA OSPEDALIERA L. SACCO (A.O. DI RILIEVO ...,Italy,Non-Commercial
1,2004-000534-36,SK,Laboratoires Expanscience,France,Commercial
2,2017-004497-33,DK,Copenhagen University Hospital at Hvidovre,Denmark,Non-Commercial
3,2009-010198-19,SE,Novartis Pharma Services AG,Switzerland,Commercial
4,2017-000048-17,DK,Novo Nordisk A/S,Denmark,Commercial


In [4]:
norm = pd.read_excel('normalized_sponsor_names_jan.xlsx')
trials_csv = pd.read_csv('trials_jan.csv')
trials = trials_csv[['trial_id', 'number_of_countries', 'has_results', 'results_expected']].drop_duplicates()

In [5]:
sample = spon_df[(spon_df.sponsor_country == 'Austria')].reset_index(drop=True)
sample1 = sample.drop_duplicates()

In [6]:
merged = sample.merge(trials, on='trial_id', how='left')
merged['name_of_sponsor_lower'] = merged.sponsor_name.str.lower().str.replace(' ', '').str.replace('[^\w\s]','')

In [7]:
norm_s = norm[['name_of_sponsor', 'normalized_name']].reset_index(drop=True)
norm_s['name_of_sponsor_lower'] = norm.name_of_sponsor.str.lower().str.replace(' ', '').str.replace('[^\w\s]','')

In [8]:
with_norm = merged.merge(norm_s, on='name_of_sponsor_lower', how='left', indicator=True)

In [9]:
with_norm[with_norm._merge == 'left_only']

,trial_id,protocol_country,sponsor_name,sponsor_country,sponsor_status,number_of_countries,has_results,results_expected,name_of_sponsor_lower,name_of_sponsor,normalized_name,_merge
799,2005-004866-17,DE,"Dept Medicine I, Paracelsus Med. Univ.\tLandes...",Austria,Non-Commercial,2.0,0.0,1.0,deptmedicineiparacelsusmeduniv\tlandeskrankena...,NaN,NaN,left_only
2533,2012-002002-46,AT,"Universitätsklinik für Zahn-, Mund- und Kiefer...",Austria,Non-Commercial,NaN,NaN,NaN,universitätsklinikfürzahnmundundkieferheilkund...,NaN,NaN,left_only


In [10]:
errors = with_norm[with_norm._merge == 'left_only'].index.values.tolist()

In [11]:
errors

[799, 2533]

In [12]:
for e in errors:
    if 'nodata' in with_norm.at[e,'name_of_sponsor_lower']:
        with_norm.at[e,'normalized_name'] = 'No Sponsor Name Given'
    elif 'universitätsklinikfürzahnmundundkieferheil' in with_norm.at[e,'name_of_sponsor_lower']:
        with_norm.at[e,'normalized_name'] = 'Medical University of Graz'
    elif 'deptmedicineiparacelsusmeduniv\tland' in with_norm.at[e,'name_of_sponsor_lower']:
        with_norm.at[e,'normalized_name'] = 'Paracelsus Medical University'

In [13]:
with_norm.head()

,trial_id,protocol_country,sponsor_name,sponsor_country,sponsor_status,number_of_countries,has_results,results_expected,name_of_sponsor_lower,name_of_sponsor,normalized_name,_merge
0,2016-001478-14,IT,Baxalta Innovations GmbH,Austria,Commercial,7.0,0.0,0.0,baxaltainnovationsgmbh,Baxalta Innovations GmbH,Baxalta Innovations GmbH,both
1,2016-001478-14,CZ,Baxalta Innovations GmbH,Austria,Commercial,7.0,0.0,0.0,baxaltainnovationsgmbh,Baxalta Innovations GmbH,Baxalta Innovations GmbH,both
2,2016-001478-14,NL,Baxalta Innovations GmbH,Austria,Commercial,7.0,0.0,0.0,baxaltainnovationsgmbh,Baxalta Innovations GmbH,Baxalta Innovations GmbH,both
3,2016-001478-14,ES,Baxalta Innovations GmbH,Austria,Commercial,7.0,0.0,0.0,baxaltainnovationsgmbh,Baxalta Innovations GmbH,Baxalta Innovations GmbH,both
4,2016-001478-14,DE,Baxalta Innovations GmbH,Austria,Commercial,7.0,0.0,0.0,baxaltainnovationsgmbh,Baxalta Innovations GmbH,Baxalta Innovations GmbH,both


In [14]:
with_norm.rename(columns={'sponsor_name': 'euctr_name'}, inplace=True)

In [15]:
almost_df = with_norm[['trial_id','sponsor_country', 'normalized_name', 'sponsor_status', 'results_expected', 
                       'has_results']]

final_df = almost_df[almost_df.has_results.notnull()].reset_index(drop=True)
final_df['results_expected'] = final_df.results_expected.astype(int)
final_df['has_results'] = final_df.has_results.astype(int)
final_df['due_with_results'] = np.where((final_df.has_results == 1) & (final_df.results_expected == 1), 1,0)
final_df['trial_count'] = 1
final_df['commercial'] = np.where(final_df['sponsor_status'] == 'Commercial',1,0)
final_df['non-commercial'] = np.where(final_df['sponsor_status'] == 'Non-Commercial',1,0)

In [16]:
final_df.dtypes

trial_id            object
sponsor_country     object
normalized_name     object
sponsor_status      object
results_expected     int64
has_results          int64
due_with_results     int64
trial_count          int64
commercial           int64
non-commercial       int64
dtype: object

In [17]:
final_df.drop_duplicates(inplace=True)
final_df.head()

,trial_id,sponsor_country,normalized_name,sponsor_status,results_expected,has_results,due_with_results,trial_count,commercial,non-commercial
0,2016-001478-14,Austria,Baxalta Innovations GmbH,Commercial,0,0,0,1,1,0
7,2010-021044-17,Austria,Medical University of Vienna,Non-Commercial,1,1,1,1,0,1
8,2010-018471-26,Austria,Boehringer Ingelheim,Commercial,1,1,1,1,1,0
13,2015-003714-24,Austria,Biomedizinische Forschungsgesellschaft m.b.H.,Commercial,0,0,0,1,1,0
14,2009-017137-22,Austria,Medical University of Graz,Non-Commercial,0,0,0,1,0,1


In [18]:
final_df.groupby('normalized_name').sum().to_csv('summary_counts.csv')

In [19]:
final_df.to_csv('combined_sponsor_info_austria.csv')